In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexByteplot"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = ResNet50(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 2048)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 2048)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  5.9min


[CV] ............................ C=0.1, score=0.903922, total= 6.0min
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.889764, total= 6.0min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.900391, total= 6.0min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.896414, total= 6.1min
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.894531, total= 6.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.891389, total= 6.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.916749, total= 6.3min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  6.6min


[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.887695, total= 6.6min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.897059, total= 6.7min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.894685, total= 6.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.895098, total= 6.6min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.902153, total= 6.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.902559, total= 6.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.908012, total= 6.8min
[CV] C

...

[CV] ................... C=1000000000.0, score=0.890411, total= 8.2min
[CV] ................... C=1000000000.0, score=0.900892, total= 7.8min
[CV] ................... C=1000000000.0, score=0.900980, total= 8.3min
[CV] ................... C=1000000000.0, score=0.896654, total= 8.0min
[CV] ................... C=1000000000.0, score=0.886811, total= 8.2min
[CV] ................... C=1000000000.0, score=0.900398, total= 7.7min
[CV] ................... C=1000000000.0, score=0.892857, total= 8.0min
[CV] ................... C=1000000000.0, score=0.902584, total= 7.9min
[CV] ................... C=1000000000.0, score=0.903066, total= 8.2min
[CV] .................. C=10000000000.0, score=0.891602, total= 4.5min
[CV] .................. C=10000000000.0, score=0.882583, total= 3.8min
[CV] .................. C=10000000000.0, score=0.903922, total= 3.3min
[CV] .................. C=10000000000.0, score=0.902584, total= 2.7min
[CV] .................. C=10000000000.0, score=0.898622, total= 3.0min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 44.5min remaining:  1.8min


[CV] .................. C=10000000000.0, score=0.883858, total= 3.2min
[CV] .................. C=10000000000.0, score=0.903386, total= 2.6min
[CV] .................. C=10000000000.0, score=0.906034, total= 3.0min
[CV] .................. C=10000000000.0, score=0.904856, total= 3.0min
[CV] .................. C=10000000000.0, score=0.889881, total= 3.0min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 44.7min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([378.51576557, 390.94173877, 478.9250968 , 549.89847529,
        527.07694683, 550.58724093, 541.8718395 , 532.84368584,
        544.31222899, 529.53995528, 500.36055663, 482.91374164,
        193.03229868]),
 'mean_score_time': array([0.07627501, 0.05813076, 0.07815495, 0.06977613, 0.06968341,
        0.06506495, 0.05293515, 0.08083115, 0.05988545, 0.06577837,
        0.04291792, 0.02926676, 0.01613874]),
 'mean_test_score': array([0.90213102, 0.90094712, 0.89384373, 0.89571823, 0.89542226,
        0.89660616, 0.89640884, 0.89581689, 0.89601421, 0.89196922,
        0.89492897, 0.89680347, 0.89670481]),
 'mean_train_score': array([0.96884582, 0.99481477, 0.99684336, 0.99532851, 0.99655763,
        0.99679914, 0.99712798, 0.99511216, 0.99670068, 0.99041493,
        0.99709486, 0.99675511, 0.99721525]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.90213, std: 0.00765, params: {'C': 0.01},
 mean: 0.90095, std: 0.00678, params: {'C': 0.1},
 mean: 0.89384, std: 0.00809, params: {'C': 1.0},
 mean: 0.89572, std: 0.00695, params: {'C': 10.0},
 mean: 0.89542, std: 0.00756, params: {'C': 100.0},
 mean: 0.89661, std: 0.00559, params: {'C': 1000.0},
 mean: 0.89641, std: 0.00500, params: {'C': 10000.0},
 mean: 0.89582, std: 0.00634, params: {'C': 100000.0},
 mean: 0.89601, std: 0.00769, params: {'C': 1000000.0},
 mean: 0.89197, std: 0.00899, params: {'C': 10000000.0},
 mean: 0.89493, std: 0.00587, params: {'C': 100000000.0},
 mean: 0.89680, std: 0.00535, params: {'C': 1000000000.0},
 mean: 0.89670, std: 0.00852, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9021
